In [1]:
!pip install tensorflow==2.9.0 
#older packet because i hab problems with json, that was pretty long journey because my first accuracy was about 11%
#also i had a lot of troubles with versions and packets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # to detect face

!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

datFile =  "/content/shape_predictor_68_face_landmarks.dat"

--2023-02-19 11:06:01--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  14.8MB/s    in 7.5s    

2023-02-19 11:06:09 (8.11 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

bunzip2: Output file /content/shape_predictor_68_face_landmarks.dat already exists.


In [ ]:
import cv2
import dlib
import os
import sys
import random
from google.colab import files
import os, shutil
from google.colab import drive
drive.mount('/content/drive')
datFile =  "/content/shape_predictor_68_face_landmarks.dat"

output_dir = '/content/drive/MyDrive/project/output_images'
size = 64

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
def relight(img, light=1, bias=0):
    w = img.shape[1]
    h = img.shape[0]
 
    for i in range(0,w):
        for j in range(0,h):
            for c in range(3):
                tmp = int(img[j,i,c]*light + bias)
                if tmp > 255:
                    tmp = 255
                elif tmp < 0:
                    tmp = 0
                img[j,i,c] = tmp
    return img

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor( 'shape_predictor_68_face_landmarks.dat')
index = 1
people = [people for people in os.listdir("/content/drive/MyDrive/project/train/")]
for people in people:
    images = [image for image in os.listdir("/content/drive/MyDrive/project/train/" + people + '/')]
    for image in images:    
        img = cv2.imread("/content/drive/MyDrive/project/train/" + people+ '/' +image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        dets = detector(img, 0)
        for i, d in enumerate(dets):
            x1 = d.top() if d.top() > 0 else 0
            y1 = d.bottom() if d.bottom() > 0 else 0
            x2 = d.left() if d.left() > 0 else 0
            y2 = d.right() if d.right() > 0 else 0           
            face = img[x1:y1,x2:y2]
            face = relight(face, random.uniform(0.5, 1.5), random.randint(-50, 50))           
            face = cv2.resize(face, (size,size))           
            cv2.imwrite(output_dir+'/'+ people +'/'+str(index)+'.jpg', face)
        index += 1
        print(index)
    print('Finished')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
Finished
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
Finished
239
240
241

In [12]:
import cv2
import os
import numpy as np
from tensorflow import keras
import tensorflow as tf

from keras.utils import np_utils
from keras import optimizers, layers, models
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

# Saving and loading model and weights
from keras.models import model_from_json
from keras.models import load_model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
# load and prepare data
people = [people for people in os.listdir("/content/drive/MyDrive/project/output_images/")]
print(people)
num_classes = 7
img_data_list = []
labels = []
valid_images = [".jpg",".gif",".png", ".jpeg", ".JPG"]

for index, person in enumerate(people):
  print(index)
  dir_path = '/content/drive/MyDrive/project/output_images/' + person
  print(dir_path)
  for img_path in os.listdir(dir_path):
    name, ext = os.path.splitext(img_path)
    if ext.lower() not in valid_images:
        continue
    img_data = cv2.imread(dir_path + '/' + img_path)
    # convert image to gray
    img_data=cv2.cvtColor(img_data, cv2.COLOR_BGR2GRAY)
    img_data_list.append(img_data)
    labels.append(index)
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
labels = np.array(labels ,dtype='int64')
img_data /= 255.0
img_data= np.expand_dims(img_data, axis=3)
Y = np_utils.to_categorical(labels, num_classes)
x,y = shuffle(img_data,Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
input_shape=img_data[0].shape
print(input_shape)
print(labels)
model = Sequential()
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu',
                 kernel_constraint=maxnorm(3)))
model.add(Dropout(0.1))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))
epochs = 30
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9,  nesterov=False, decay = decay)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


['heeseung', 'jay', 'jake', 'sunghoon', 'sunoo', 'jungwon', 'niki']
0
/content/drive/MyDrive/project/output_images/heeseung
1
/content/drive/MyDrive/project/output_images/jay
2
/content/drive/MyDrive/project/output_images/jake
3
/content/drive/MyDrive/project/output_images/sunghoon
4
/content/drive/MyDrive/project/output_images/sunoo
5
/content/drive/MyDrive/project/output_images/jungwon
6
/content/drive/MyDrive/project/output_images/niki
(64, 64, 1)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2

In [8]:
model.save_weights("/content/model.h5")
from google.colab import files
files.download("/content/model.h5")

print("Saved model to disk")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved model to disk


In [10]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
